In [1]:
from semantic_mpc_interface import (
    LoadModel,
    get_thermostat_data,
    HPFlexSurvey,
    convert_units,
    SHACLHandler,
    inline_shapes,
    Graph,
    get_uri_name
)
from semantic_mpc_interface.utils import query_to_df
from semantic_mpc_interface.namespaces import BRICK, RDF, S223, A, HPFS, REF
from rdflib import URIRef, Literal
from buildingmotif import BuildingMOTIF, get_building_motif
from buildingmotif.dataclasses import Library
import csv
from pyshacl.rdfutil import clone
# SELECT ONTOLOGY 
ontology = 'brick'
# base path
base_path = f'{ontology}'

CRITICAL:root:Install the 'bacnet-ingress' module, e.g. 'pip install buildingmotif[bacnet-ingress]'


In [2]:
# switching out external references
g = Graph()
g.parse('post-processed-model.ttl', format = 'ttl')
remove_triples = []
for s, ref in g.subject_objects(REF.hasExternalReference):
    for name in g.objects(ref, REF.name):
        normal_type = name.split('_')[-2]
        normal_ext_ref = URIRef(str(s) + '_normal')
        g.add((normal_ext_ref, REF.name, Literal(normal_type)))
        g.add((s, REF.hasExternalReference, normal_ext_ref))
        remove_triples.append((s, REF.hasExternalReference, ref))
for triple in remove_triples:
    g.remove(triple)
g.serialize('post-processed-model-normal.ttl', format = 'ttl')

<Graph identifier=Necf9789e37b5442183d52110998130e1 (<class 'rdflib.graph.Graph'>)>

# Writing SHACL and Running Inference

In [3]:
# Please disregard excessive outputs (logging and warnings) from bmotif
import logging
logging.disable(logging.CRITICAL)
import warnings
warnings.filterwarnings("ignore")

Generating shapes "mints" our templates into an ontology 

The templates describe how certain things should be semantically represented in our graph. 
We can fill the templates to create these semantic representations.
However, using the SHACLHandler we can use the rules of the s223 or brick ontologies to invert the templates, and instead create validation rules looking for things represented like our templates.

The templates will be added as classes to a shacl graph, currently using the namespace HPFS (urn:hpflex/shapes), and we will infer what things in the graph match our templates. 

In [4]:
handler = SHACLHandler(ontology=ontology, template_dir = '/Users/lazlopaul/Desktop/223p/experiments/simbuild-semantic-25/templates')
# Generate shapes
handler.generate_shapes()
# Save shapes
handler.save_shapes(f'{base_path}/shapes.ttl')
# also save more human readable shapes
inline_shapes(handler.shapes_graph).serialize(f"{base_path}/inlined_shapes.ttl")


BuildingMOTIF does not exist, instantiating: 


<Graph identifier=N70faed37bc134110aac2163238db1669 (<class 'rdflib.graph.Graph'>)>

In [5]:
g = Graph()
g.parse('post-processed-model-normal.ttl', format = 'ttl')
og = clone.clone_graph(g)
# run if creating new inferred graph, take ~10 seconds
# inferred_graph = handler.infer(g)
inferred_graph = Graph()
inferred_graph.parse('inferred_graph.ttl', format = 'ttl')

<Graph identifier=N9a71cb11f4c044f3848ad6f0bd473d47 (<class 'rdflib.graph.Graph'>)>

Here we can see that many classes have been inferred based on our templates. 
All the zone temperatures will be labeled with the class HPFS:temp. Same for our zone temperatures.

We also infer some basic information, such as has-point. The semantic_model_builder currently uses these templates as the interface between the HPFS ontology and another ontology (i.e. brick or S223). This was done as an experimental approach for ontology alignment, but means that we must create some boilerplate templates to load data into objects.

In [6]:
for triple in (inferred_graph - og):
    if ('Ref' in str(triple[0])) or ('has-point' in str(triple[1])):
        continue 
    else:
        print(triple)

(rdflib.term.URIRef('https://BESTESTAir.urn#hvac_reaAhu_TSup'), rdflib.term.URIRef('https://brickschema.org/schema/Brick/ref#hasExternalReference'), rdflib.term.URIRef('https://BESTESTAir.urn#hvac_reaAhu_TSup_Reference'))
(rdflib.term.URIRef('https://BESTESTAir.urn#hvac_oveZonSupEas_TZonCooSet'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('urn:hpflex/shapes#cooset'))
(rdflib.term.URIRef('https://BESTESTAir.urn#hvac_reaZonNor_V_flow'), rdflib.term.URIRef('https://brickschema.org/schema/Brick/ref#hasExternalReference'), rdflib.term.URIRef('https://BESTESTAir.urn#hvac_reaZonNor_V_flow_Reference'))
(rdflib.term.URIRef('https://BESTESTAir.urn#hvac_reaZonCor_V_flow'), rdflib.term.URIRef('https://brickschema.org/schema/Brick/ref#hasExternalReference'), rdflib.term.URIRef('https://BESTESTAir.urn#hvac_reaZonCor_V_flow_Reference'))
(rdflib.term.URIRef('https://BESTESTAir.urn#hvac_oveZonSupEas_TZonHeaSet'), rdflib.term.URIRef('https://brickschema.org

In [7]:
list(inferred_graph[:A:HPFS['heaset']])

[rdflib.term.URIRef('https://BESTESTAir.urn#hvac_oveZonSupCor_TZonHeaSet'),
 rdflib.term.URIRef('https://BESTESTAir.urn#hvac_oveZonSupEas_TZonHeaSet'),
 rdflib.term.URIRef('https://BESTESTAir.urn#hvac_oveZonSupNor_TZonHeaSet'),
 rdflib.term.URIRef('https://BESTESTAir.urn#hvac_oveZonSupSou_TZonHeaSet'),
 rdflib.term.URIRef('https://BESTESTAir.urn#hvac_oveZonSupWes_TZonHeaSet')]

In [8]:
# Currently we load graphs from files for LoadModel, so must serialize this
inferred_graph.serialize(f"inferred_graph.ttl")

<Graph identifier=N9a71cb11f4c044f3848ad6f0bd473d47 (<class 'rdflib.graph.Graph'>)>

# Demonstrating Loading Data 

LoadModel takes a filepath to a graph, a dictionary of mapping the objects you want to create, to the templates you want to create them with, the ontology used, and optionally a directory of templates.

site_info will contain dictionary of our desired templates. Each template may contain some other entities or properties. Entities and properties will be named arbitrarily based on the name of their templates. Properties are defined so that the have a value (a number, string, or external reference) and a unit. 

We want to get python objects representing each of our zones, so we will create objects based on our zone templates. These zone templates link to 3 properties for the temperature and setpoints. Each of these properties will have a value that points to the BOPTest external reference, and they will have a unit. NOTE: the units are currently incorrectly using the qudt namespace, rather than the unit namespace

In [ ]:
loader = LoadModel(f"inferred_graph.ttl", 
                   template_dict={'zones': 'zone'}, 
                   ontology = ontology,
                   template_dir = 'templates')
site_info = loader.get_all_building_objects()

In [ ]:
site_info

{'zones': [zone(name='https://BESTESTAir.urn#flo_Wes', heaset=Value(value=hvac_oveZonSupWes_TZonHeaSet_u, unit='http://qudt.org/schema/qudt/K'), cooset=Value(value=hvac_oveZonSupWes_TZonCooSet_u, unit='http://qudt.org/schema/qudt/K'), temp=Value(value=hvac_reaZonWes_TZon_y , unit='http://qudt.org/schema/qudt/K'), ),
  zone(name='https://BESTESTAir.urn#flo_Sou', heaset=Value(value=hvac_oveZonSupSou_TZonHeaSet_u, unit='http://qudt.org/schema/qudt/K'), cooset=Value(value=hvac_oveZonSupSou_TZonCooSet_u, unit='http://qudt.org/schema/qudt/K'), temp=Value(value=hvac_reaZonSou_TZon_y , unit='http://qudt.org/schema/qudt/K'), ),
  zone(name='https://BESTESTAir.urn#flo_Nor', heaset=Value(value=hvac_oveZonSupNor_TZonHeaSet_u, unit='http://qudt.org/schema/qudt/K'), cooset=Value(value=hvac_oveZonSupNor_TZonCooSet_u, unit='http://qudt.org/schema/qudt/K'), temp=Value(value=hvac_reaZonNor_TZon_y , unit='http://qudt.org/schema/qudt/K'), ),
  zone(name='https://BESTESTAir.urn#flo_Eas', heaset=Value(value

In [ ]:
site_info['zones']

[zone(name='https://BESTESTAir.urn#flo_Wes', heaset=Value(value=hvac_oveZonSupWes_TZonHeaSet_u, unit='http://qudt.org/schema/qudt/K'), cooset=Value(value=hvac_oveZonSupWes_TZonCooSet_u, unit='http://qudt.org/schema/qudt/K'), temp=Value(value=hvac_reaZonWes_TZon_y , unit='http://qudt.org/schema/qudt/K'), ),
 zone(name='https://BESTESTAir.urn#flo_Sou', heaset=Value(value=hvac_oveZonSupSou_TZonHeaSet_u, unit='http://qudt.org/schema/qudt/K'), cooset=Value(value=hvac_oveZonSupSou_TZonCooSet_u, unit='http://qudt.org/schema/qudt/K'), temp=Value(value=hvac_reaZonSou_TZon_y , unit='http://qudt.org/schema/qudt/K'), ),
 zone(name='https://BESTESTAir.urn#flo_Nor', heaset=Value(value=hvac_oveZonSupNor_TZonHeaSet_u, unit='http://qudt.org/schema/qudt/K'), cooset=Value(value=hvac_oveZonSupNor_TZonCooSet_u, unit='http://qudt.org/schema/qudt/K'), temp=Value(value=hvac_reaZonNor_TZon_y , unit='http://qudt.org/schema/qudt/K'), ),
 zone(name='https://BESTESTAir.urn#flo_Eas', heaset=Value(value=hvac_oveZonS

In [ ]:
for zone in site_info['zones']:
    print("ZONE NAME IS: ", zone.name)
    print("TEMP IS: ", zone.temp)
    print("TEMP DATA IS AT EXTERNAL REFERENCE: ", zone.temp.value)
    print("HEATING SETPOINT IS: ", zone.heaset)
    print("COOLING SEPTIONT IS: ", zone.cooset)
    print("Setpoint DATA is at External References: ", zone.heaset.value,', ', zone.cooset.value)
    print('-'* 20)

ZONE NAME IS:  https://BESTESTAir.urn#flo_Wes
TEMP IS:  Value(value=hvac_reaZonWes_TZon_y , unit='http://qudt.org/schema/qudt/K')
TEMP DATA IS AT EXTERNAL REFERENCE:  hvac_reaZonWes_TZon_y 
HEATING SETPOINT IS:  Value(value=hvac_oveZonSupWes_TZonHeaSet_u, unit='http://qudt.org/schema/qudt/K')
COOLING SEPTIONT IS:  Value(value=hvac_oveZonSupWes_TZonCooSet_u, unit='http://qudt.org/schema/qudt/K')
Setpoint DATA is at External References:  hvac_oveZonSupWes_TZonHeaSet_u ,  hvac_oveZonSupWes_TZonCooSet_u
--------------------
ZONE NAME IS:  https://BESTESTAir.urn#flo_Sou
TEMP IS:  Value(value=hvac_reaZonSou_TZon_y , unit='http://qudt.org/schema/qudt/K')
TEMP DATA IS AT EXTERNAL REFERENCE:  hvac_reaZonSou_TZon_y 
HEATING SETPOINT IS:  Value(value=hvac_oveZonSupSou_TZonHeaSet_u, unit='http://qudt.org/schema/qudt/K')
COOLING SEPTIONT IS:  Value(value=hvac_oveZonSupSou_TZonCooSet_u, unit='http://qudt.org/schema/qudt/K')
Setpoint DATA is at External References:  hvac_oveZonSupSou_TZonHeaSet_u ,  

# Implementation in CDL

So, what goes into the CDL file? 

We have multiple options:

1. __Getting bindings using simple relations:__ 
    Say what points should be used for each controller, referencing the "minted ontology"/SHACL shapes from your templates (e.g. hpfs:temp, hpfs:heaset, hpfs:cooset.) We will have to create some predicates to do this (like hpfs:binds), but I think this may be the clearer.

2. __Representing controllers in semantic model:__
    Create templates representing the controller. Reference these templates in the CDL. Instantiate the templates to create s223 models of the controller for each zone and connect them to the properties in the model. We add the controller models to our graph. However, we will still have to do some querying probably to get the necessary data out for Normal. The expedient option may make more sense for now, but this was the option we landed on doing C2S

In [ ]:
# Opt 1: just say what points should map to each input/output (and what template we should use for the query)
# ctrl:controller, ctrl:heaset_u, etc. would probably be written as <cdl_instance_name> in the modelica file. 
data = """
@prefix bldg: <https://BESTESTAir.urn#> .
@prefix boptestrules: <https://boptest-rules.urn#> .
@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix hpfs: <urn:hpflex/shapes#> .
@prefix quantitykind: <http://qudt.org/vocab/quantitykind/> .
@prefix qudt: <http://qudt.org/schema/qudt/> .
@prefix ref: <https://brickschema.org/schema/Brick/ref#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

@prefix s223: <http://data.ashrae.org/standard223#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .

@prefix ctrl: <urn:mycontroller.urn#> .

ctrl:controller hpfs:controls hpfs:zone .

ctrl:heaset_u hpfs:binds hpfs:heaset .

ctrl:cooset_u hpfs:binds hpfs:cooset .

ctrl:temp_u hpfs:binds hpfs:temp .

ctrl:temp_y hpfs:binds hpfs:temp . 
"""

In [ ]:
# Get our graph and the name of the template representing what we intend to control
ctrl_graph = Graph()
ctrl_graph.parse(data=data, format='turtle')
target_template_uri = list(ctrl_graph[:HPFS.controls:])[0][1]
target_template_name = get_uri_name(ctrl_graph, target_template_uri)
print('looking for template: ',target_template_name)

# Load the graph and get all the opjects matching this template, as before
loader = LoadModel(f"inferred_graph.ttl", 
                   template_dict={'control_target': target_template_name}, 
                   ontology = ontology,
                   template_dir = 'templates')
site_data = loader.get_all_building_objects()

# get targets of our controls
control_targets = site_data['control_target']

looking for template:  zone


In [ ]:
# get bindings of the controller 
bindings = list(ctrl_graph[:HPFS.binds:])
for target in control_targets:
    print(f"for Target: {target.name}")
    for binding in bindings:
        binding_name = binding[1].split('#')[-1]
        print(f'Point {binding[0]} binds {getattr(target, binding_name)}')
    print('-'*20)



for Target: https://BESTESTAir.urn#flo_Wes
Point urn:mycontroller.urn#heaset_u binds Value(value=hvac_oveZonSupWes_TZonHeaSet_u, unit='http://qudt.org/schema/qudt/K')
Point urn:mycontroller.urn#cooset_u binds Value(value=hvac_oveZonSupWes_TZonCooSet_u, unit='http://qudt.org/schema/qudt/K')
Point urn:mycontroller.urn#temp_u binds Value(value=hvac_reaZonWes_TZon_y , unit='http://qudt.org/schema/qudt/K')
Point urn:mycontroller.urn#temp_y binds Value(value=hvac_reaZonWes_TZon_y , unit='http://qudt.org/schema/qudt/K')
--------------------
for Target: https://BESTESTAir.urn#flo_Sou
Point urn:mycontroller.urn#heaset_u binds Value(value=hvac_oveZonSupSou_TZonHeaSet_u, unit='http://qudt.org/schema/qudt/K')
Point urn:mycontroller.urn#cooset_u binds Value(value=hvac_oveZonSupSou_TZonCooSet_u, unit='http://qudt.org/schema/qudt/K')
Point urn:mycontroller.urn#temp_u binds Value(value=hvac_reaZonSou_TZon_y , unit='http://qudt.org/schema/qudt/K')
Point urn:mycontroller.urn#temp_y binds Value(value=hva

## Connecting to CDL - Method that also creates and adds models of the controller (uncomplete)

In [ ]:
# Let's try option 2, a more detailed option 
data = """
@prefix bldg: <https://BESTESTAir.urn#> .
@prefix boptestrules: <https://boptest-rules.urn#> .
@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix hpfs: <urn:hpflex/shapes#> .
@prefix quantitykind: <http://qudt.org/vocab/quantitykind/> .
@prefix qudt: <http://qudt.org/schema/qudt/> .
@prefix ref: <https://brickschema.org/schema/Brick/ref#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

@prefix s223: <http://data.ashrae.org/standard223#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .

@prefix ctrl: <urn:mycontroller.urn#> .

ctrl:controller a hpfs:zone-temp-controller ; 
    s223:hasInput ctrl:heaset_u, ctrl:cooset_u, ctrl:temp_u ;
    s223:hasOutput ctrl:temp_y .

ctrl:heaset_u a hpfs:heaset_u .
ctrl:cooset_u a hpfs:cooset_u .
ctrl:temp_u a hpfs:temp_u .
ctrl:temp_y a hpfs:temp_y . 
"""

In [ ]:
# the graph that represents the controller (exported from modelica)
ctrl_graph = Graph()
ctrl_graph.parse(data=data, format='turtle')

<Graph identifier=Na48e49a9b1b7416199fe572262802856 (<class 'rdflib.graph.Graph'>)>

In [ ]:
def remove_optional_deps(template):
    opt_args = template.optional_args
    for opt_arg in opt_args:
        template.remove_dependency(template.dependency_for_parameter(opt_arg))

In [ ]:
# We will instantiate the controller for each zone and add it to a new graph
ctrl_bldg_graph = Graph()
for zone in site_info['zones']:
    # ctrl graph indicates the names of each parameter and what templates are used
    for ctrl_point, klass in ctrl_graph[:A:]:
        # get the template
        template_name = get_uri_name(ctrl_graph, klass)
        template = loader.library.get_template_by_name(template_name)
        param_values = {}
        print('ctrl_point: ', ctrl_point)
        for p in template.parameters:
            print('Param: ', p)
            if p == 'name':
                param_values[p] = ctrl_point
                continue
            param_type = template.dependency_for_parameter(p)
            print('param_type: ',param_type.name)
            point = ctrl_graph.value(None, A, HPFS[param_type.name])
            print('matching point: ',point)
        print('-'* 20)
        
        # loader.list_available_templates()

ctrl_point:  urn:mycontroller.urn#controller
Param:  temp_u
param_type:  temp_u
matching point:  urn:mycontroller.urn#temp_u
Param:  heaset_u
param_type:  heaset_u
matching point:  urn:mycontroller.urn#heaset_u
Param:  temp_y
param_type:  temp_y
matching point:  urn:mycontroller.urn#temp_y
Param:  name
Param:  cooset_u
param_type:  cooset_u
matching point:  urn:mycontroller.urn#cooset_u
--------------------
ctrl_point:  urn:mycontroller.urn#heaset_u
Param:  heaset
param_type:  heaset
matching point:  None
Param:  name
--------------------
ctrl_point:  urn:mycontroller.urn#cooset_u
Param:  cooset
param_type:  cooset
matching point:  None
Param:  name
--------------------
ctrl_point:  urn:mycontroller.urn#temp_u
Param:  temp
param_type:  temp
matching point:  None
Param:  name
--------------------
ctrl_point:  urn:mycontroller.urn#temp_y
Param:  temp
param_type:  temp
matching point:  None
Param:  name
--------------------
ctrl_point:  urn:mycontroller.urn#controller
Param:  temp_u
param

In [ ]:
template = loader.library.get_template_by_name('zone-temp-controller')
template.parameters

{'cooset_u', 'heaset_u', 'name', 'temp_u', 'temp_y'}

In [ ]:
template.dependency_for_parameter('temp_u')

Template(_id=10, _name='temp_u', body=<Graph identifier=00825b8d-0c9d-4cdb-85d6-63f68a3e3602 (<class 'rdflib.graph.Graph'>)>, optional_args=['temp'], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x11e4168d0>)

In [ ]:
template.dependency_for_parameter('cooset_u').name

'cooset_u'

In [ ]:
d = template.get_dependencies()[0]

In [ ]:
dir(template)

['__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_bm',
 '_id',
 '_name',
 'add_dependency',
 'all_parameters',
 'body',
 'check_dependencies',
 'defining_library',
 'dependency_for_parameter',
 'dependency_parameters',
 'evaluate',
 'fill',
 'find_subgraphs',
 'generate_csv',
 'generate_spreadsheet',
 'get_dependencies',
 'id',
 'in_memory_copy',
 'inline_dependencies',
 'library_dependencies',
 'load',
 'name',
 'optional_args',
 'parameter_counts',
 'parameters',
 'remove_dependency',
 'to_inline',
 'transitive_parameters']

In [ ]:
template = loader.library.get_template_by_name('heaset_u')

In [ ]:
template.optional_args

['heaset']

In [ ]:
template.dependency_for_parameter('heaset')

Template(_id=6, _name='heaset', body=<Graph identifier=3733bafb-1802-442c-9af3-742ad8ed3d5c (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x11e4168d0>)

In [ ]:
template.inline_dependencies().body.print()

@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix ns1: <https://brickschema.org/schema/Brick/ref#> .
@prefix ns2: <http://data.ashrae.org/standard223#> .
@prefix ns3: <http://qudt.org/schema/qudt/> .

<urn:___param___#name> a ns2:FunctionInput ;
    ns2:uses <urn:___param___#heaset> .

<urn:___param___#heaset> a brick:Zone_Air_Heating_Temperature_Setpoint ;
    ns3:hasUnit ns3:K ;
    ns1:hasExternalReference <urn:___param___#heaset-ref> .

<urn:___param___#heaset-ref> a ns1:BOPTestReference .


